### Importing modules

In [1]:
# import time
# import os
import sys

# import modules
import pandas as pd
import torch
import torch.optim as optim
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid

from training.test import testing
from training.train import training
from utils.miscellaneous import create_folder_structure_MLPvsGNN
from utils.miscellaneous import initalize_random_generators
from utils.miscellaneous import read_config

### Parse configuration file + initializations

In [2]:
# read config files
cfg = read_config("configs/config_MLP_vs_GNN.yaml")

# create folder for results
exp_name = cfg['exp_name']
data_folder = cfg['data_folder']
results_folder = create_folder_structure_MLPvsGNN(cfg, parent_folder='./experiments')

all_wdn_names = cfg['networks']
initalize_random_generators(cfg, count=0)

# initialize pytorch device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device ='cpu'
print(device)

Creating folder: ./experiments/GNN_8000
cuda:0


In [3]:
num_epochs = cfg['trainParams']['num_epochs']
batch_size = cfg['trainParams']['batch_size']
alpha = cfg['lossParams']['alpha']
res_columns = ['train_loss', 'valid_loss','test_loss', 'r2_train', 'r2_valid',
               'r2_test','total_params','total_time','num_epochs']

### Functions

In [4]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator,TransformerMixin

class PowerLogTransformer(BaseEstimator,TransformerMixin):
    def __init__(self,log_transform=False,power=4,reverse=True):
        if log_transform == True:
            self.log_transform = log_transform
            self.power = None
        else:
            self.power = power
            self.log_transform = None
        self.reverse=reverse
        self.max_ = None
        self.min_ = None
        
    def fit(self,X,y=None):        
        self.max_ = np.max(X)
        self.min_ = np.min(X)        
        return self
    
    def transform(self,X):
        if self.log_transform==True:
            if self.reverse == True:
                return np.log1p(self.max_-X)
            else:
                return np.log1p(X-self.min_)
        else:
            if self.reverse == True:
                return (self.max_-X)**(1/self.power )
            else:
                return (X-self.min_)**(1/self.power )
            
    def inverse_transform(self,X):
        if self.log_transform==True:
            if self.reverse == True:
                return (self.max_ - np.exp(X))
            else:
                return (np.exp(X) + self.min_)
        else:
            if self.reverse == True:
                return (self.max_ - X**self.power )               
            else:
                return (X**self.power + self.min_)               
    
class GraphNormalizer:
    def __init__(self, x_feat_names=['elevation','base_demand','base_head'],
                 ea_feat_names=['diameter','length','roughness'], output='pressure'):        
        # store 
        self.x_feat_names = x_feat_names
        self.ea_feat_names = ea_feat_names
        self.output = output
        
        # create separate scaler for each feature (can be improved, e.g., you can fit a scaler for multiple columns)
        self.scalers = {}
        for feat in self.x_feat_names:
            if feat == 'elevation':
                self.scalers[feat] = PowerLogTransformer(log_transform=True,reverse=False)
            else:
                self.scalers[feat] = MinMaxScaler()
        self.scalers[output] = PowerLogTransformer(log_transform=True,reverse=True)
        for feat in self.ea_feat_names:
            if feat == 'length':
                self.scalers[feat] = PowerLogTransformer(log_transform=True,reverse=False)
            else:
                self.scalers[feat] = MinMaxScaler()            
            
    def fit(self, graphs):
        ''' Fit the scalers on an array of x and ea features
        '''
        x, y, ea = from_graphs_to_pandas(graphs)
        for ix, feat in enumerate(self.x_feat_names):
            self.scalers[feat] = self.scalers[feat].fit(x[:,ix].reshape(-1,1))
        self.scalers[self.output] = self.scalers[self.output].fit(y.reshape(-1,1))
        for ix, feat in enumerate(self.ea_feat_names):
            self.scalers[feat] = self.scalers[feat].fit(ea[:,ix].reshape(-1,1))        
        return self

    def transform(self, graph):
        ''' Transform graph based on normalizer
        '''
        graph = graph.clone()
        for ix, feat in enumerate(self.x_feat_names):
            temp = graph.x[:,ix].numpy().reshape(-1,1)
            graph.x[:,ix] = torch.tensor(self.scalers[feat].transform(temp).reshape(-1))
        for ix, feat in enumerate(self.ea_feat_names):
            temp = graph.edge_attr[:,ix].numpy().reshape(-1,1)
            graph.edge_attr[:,ix] = torch.tensor(self.scalers[feat].transform(temp).reshape(-1))
        graph.y = torch.tensor(self.scalers[self.output].transform(graph.y.numpy().reshape(-1,1)).reshape(-1))                   
        return graph

    def inverse_transform(self, graph):
        ''' Perform inverse transformation to return original features
        '''
        graph = graph.clone()
        for ix, feat in enumerate(self.x_feat_names):
            temp = graph.x[:,ix].numpy().reshape(-1,1)
            graph.x[:,ix] = torch.tensor(self.scalers[feat].inverse_transform(temp).reshape(-1))
        for ix, feat in enumerate(self.ea_feat_names):
            temp = graph.edge_attr[:,ix].numpy().reshape(-1,1)
            graph.edge_attr[:,ix] = torch.tensor(self.scalers[feat].inverse_transform(temp).reshape(-1))
        graph.y = torch.tensor(self.scalers[self.output].inverse_transform(graph.y.numpy().reshape(-1,1)).reshape(-1))                   
        return graph
            
    def transform_array(self,z,feat_name):
        '''
            This is for MLP dataset; it can be done better (the entire thing, from raw data to datasets)
        '''
        return torch.tensor(self.scalers[feat_name].transform(z).reshape(-1))
        
    def inverse_transform_array(self,z,feat_name):
        '''
            This is for MLP dataset; it can be done better (the entire thing, from raw data to datasets)
        '''
        return torch.tensor(self.scalers[feat_name].inverse_transform(z).reshape(-1))

def from_graphs_to_pandas(graphs, l_x=3, l_ea=3):
    x = []
    y = []
    ea = []
    for i, graph in enumerate(graphs):
        x.append(graph.x.numpy())
        y.append(graph.y.reshape(-1,1).numpy())
        ea.append(graph.edge_attr.numpy())     
    return np.concatenate(x,axis=0),np.concatenate(y,axis=0),np.concatenate(ea,axis=0)

In [5]:
import pickle
import numpy as np

# constant indexes for node and edge features
ELEVATION_INDEX = 0
BASEDEMAND_INDEX = 1
BASEHEAD_INDEX = 2
DIAMETER_INDEX = 0
LENGTH_INDEX = 1
ROUGHNESS_INDEX = 2

def load_raw_dataset(wdn_name, data_folder):
    '''
    Load tra/val/data for a water distribution network datasets
    -------
    wdn_name : string
        prefix of pickle files to open
    data_folder : string
        path to datasets
    '''

    data_tra = pickle.load(open(f'{data_folder}/train/{wdn_name}.p', "rb"))
    data_val = pickle.load(open(f'{data_folder}/valid/{wdn_name}.p', "rb"))
    data_tst = pickle.load(open(f'{data_folder}/test/{wdn_name}.p', "rb"))

    return data_tra, data_val, data_tst

def create_dataset(database, normalizer=None, HW_rough_minmax=[60, 150],add_virtual_reservoirs=False, output='pressure'):
    '''
    Creates working datasets dataset from the pickle databases	
    ------    
    database : list
        each element in the list is a pickle file containing Data objects
    normalization: dict
        normalize the dataset using mean and std
    '''
    # Roughness info (Hazen-Williams) / TODO: remove the hard_coding
    minR = HW_rough_minmax[0]
    maxR = HW_rough_minmax[1]

    graphs = []

    for i in database:
        graph = torch_geometric.data.Data()

        # Node attributes
        graph.x = torch.stack((i.elevation+i.base_head, i.base_demand, i.type_1H), dim=1).float()

        # Position and ID
        graph.ID = i.ID

        # Edge index (Adjacency matrix)
        graph.edge_index = i.edge_index

        # Edge attributes
        diameter = i.diameter
        length = i.length
        roughness = i.roughness
        graph.edge_attr = torch.stack((diameter, length, roughness), dim=1).float()

        # Graph output (head)
        if output == 'head':
            raise ValueError('Not yet implemented')
        else:
            graph.y = i.pressure[i.type_1H == 0].reshape(-1, 1)

        
        # normalization
        if normalizer is not None:
            graph = normalizer.transform(graph)

        graphs.append(graph)
    return graphs

def create_dataset_MLP_from_graphs(graphs, features=['diameter', 'base_demand', 'roughness'],no_res_out=True):
    '''
    TO DO
    '''

    # index edges to avoid duplicates: this considers all graphs to be UNDIRECTED!
    ix_edge = graphs[0].edge_index.numpy().T
    ix_edge = (ix_edge[:, 0] < ix_edge[:, 1])
    
    # position of reservoirs
    ix_res = graphs[0].x[:,BASEHEAD_INDEX].numpy()>0

    for ix_feat, feature in enumerate(features):
        for ix_item, item in enumerate(graphs):
            if feature == 'diameter':                
                x_ = item.edge_attr[ix_edge,DIAMETER_INDEX]
            elif feature == 'roughness':
                x_ = item.edge_attr[ix_edge,ROUGHNESS_INDEX]
            elif feature == 'base_demand':
                # remove reservoirs
                x_ = item.x[~ix_res,BASEDEMAND_INDEX]
            else:
                raise ValueError(f'Feature {feature} not supported.')
            if ix_item == 0:
                x = x_
            else:
                x = torch.cat((x, x_), dim=0)
        if ix_feat == 0:
            X = x.reshape(len(graphs), -1)
        else:
            X = torch.cat((X, x.reshape(len(graphs), -1)), dim=1)

    for ix_item, item in enumerate(graphs):
        # remove reservoirs from y as well
        if ix_item == 0:
            if no_res_out == True:
                y = item.y
            else:
                y = item.y[~ix_res]
        else:
            if no_res_out == True:
                y = torch.cat((y, item.y), dim=0)
            else:
                y = torch.cat((y, item.y[~ix_res]), dim=0)
    y = y.reshape(len(graphs), -1)

    return torch.utils.data.TensorDataset(X, y)    
    # return X,y,ix_res

In [6]:
# Libraries
import torch.nn as nn
import torch_geometric
import torch_geometric.nn.models
from torch import Tensor
from torch.nn import Dropout
from torch_geometric.nn import ChebConv, MessagePassing
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_geometric.nn.inits import reset
from typing import Union


class NNConvEmbed(MessagePassing):
    def __init__(self, x_num, ea_num, emb_channels, aggr, dropout_rate=0):
        super(NNConvEmbed, self).__init__(aggr=aggr)

        self.x_num = x_num
        self.ea_num = ea_num
        self.emb_channels = emb_channels
        self.nn = Sequential(Linear(2 * x_num + ea_num, emb_channels), Dropout(p=dropout_rate), ReLU())
        self.aggr = aggr
        self.reset_parameters()

    def reset_parameters(self):
        reset(self.nn)

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                edge_attr: OptTensor = None, size: Size = None) -> Tensor:
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=size)

        return out

    def message(self, x_i, x_j, edge_attr):
        z = torch.cat([x_i, x_j, edge_attr], dim=-1)
        return self.nn(z)

    def __repr__(self):
        return '{}(aggr="{}", nn={})'.format(self.__class__.__name__, self.aggr, self.nn)


class GNN_ChebConv(nn.Module):
    def __init__(self, hid_channels, edge_features, node_features, edge_channels=32, dropout_rate=0, CC_K=2,
                 emb_aggr='max', depth=2, normalize=True):
        super(GNN_ChebConv, self).__init__()
        self.hid_channels = hid_channels
        self.dropout = dropout_rate
        self.normalize = normalize

        # embedding of node/edge features with NN
        self.embedding = NNConvEmbed(node_features, edge_features, edge_channels, aggr=emb_aggr)

        # CB convolutions (with normalization)
        self.convs = nn.ModuleList()
        for i in range(depth):
            if i == 0:
                self.convs.append(ChebConv(edge_channels, hid_channels, CC_K, normalization='sym'))
            else:
                self.convs.append(ChebConv(hid_channels, hid_channels, CC_K, normalization='sym'))

        # output layer (so far only a 1 layer MLP, make more?)
        if depth == 0:
            self.lin = Linear(edge_channels, 1)
        else:
            self.lin = Linear(hid_channels, 1)

    def forward(self, data):

        # retrieve model device (for LayerNorm to work)
        device = next(self.parameters()).device

        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr

        # 1. Pre-process data (nodes and edges) with MLP
        x = self.embedding(x=x, edge_index=edge_index, edge_attr=edge_attr)

        # 2. Do convolutions
        for i in range(len(self.convs)):
            x = self.convs[i](x=x, edge_index=edge_index)
            if self.normalize:
                x = nn.LayerNorm(self.hid_channels, eps=1e-5, device=device)(x)
            x = F.dropout(x, self.dropout, training=self.training)
            x = nn.ReLU()(x)

        # 3. Output
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lin(x)

        # Mask over storage nodes (which have pressure=0)
        x = x[data.x[:,2]<1,0]

        return x

In [7]:
# Libraries
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import Sequential, Linear, ReLU

class MLP(nn.Module):
    def __init__(self, hid_channels, num_inputs, num_outputs, num_layers=2, dropout_rate=0):
        super(MLP, self).__init__()
        torch.manual_seed(42)
        self.hid_channels = hid_channels
        self.dropout_rate = dropout_rate
        
        layers = [Linear(num_inputs, hid_channels),
                  nn.ReLU(),
                  nn.Dropout(self.dropout_rate)]
        
        for l in range(num_layers-1):
            layers += [Linear(hid_channels, hid_channels),
                       nn.ReLU(),
                       nn.Dropout(self.dropout_rate)]
            
        # layers += [Linear(hid_channels, num_outputs),nn.Sigmoid()]
        layers += [Linear(hid_channels, num_outputs)]
        
        self.main = nn.Sequential(*layers)
        
    def forward(self, x):
        
        x = self.main(x)
        
        return x

### Running experiments

In [8]:
for ix_wdn, wdn in enumerate(all_wdn_names):
    print(f'\nWorking with {wdn}, network {ix_wdn+1} of {len(all_wdn_names)}')
    
    # retrieve wntr data
    tra_database, val_database, tst_database = load_raw_dataset(wdn, data_folder)
    # reduce training data    
    # tra_database = tra_database[:int(len(tra_database)*cfg['tra_prc'])]
    if cfg['tra_num'] < len(tra_database):
        tra_database = tra_database[:cfg['tra_num']]
        
    # remove PES anomaly
    if wdn == 'PES':
        if len(tra_database)>4468:
            del tra_database[4468]
            print('Removed PES anomaly')
            print('Check',tra_database[4468].pressure.mean())
    
    # get GRAPH datasets    # later on we should change this and use normal scalers from scikit
    tra_dataset = create_dataset(tra_database)
    gn = GraphNormalizer()
    gn = gn.fit(tra_dataset)
    tra_dataset = create_dataset(tra_database, normalizer=gn)
    val_dataset = create_dataset(val_database, normalizer=gn)
    tst_dataset = create_dataset(tst_database, normalizer=gn)
    node_size, edge_size = tra_dataset[0].x.size(-1), tra_dataset[0].edge_attr.size(-1)  
    # number of nodes
    # n_nodes=tra_dataset[0].x.shape[0]
    n_nodes=(1-tra_database[0].type_1H).numpy().sum() # remove reservoirs
    
    # loop through different algorithms
    for algorithm in cfg['algorithms']:
        hyperParams = cfg['hyperParams'][algorithm]
        all_combinations = ParameterGrid(hyperParams)
                        
        # dataloader
        if algorithm == 'MLP':
            # transform dataset for MLP / ad-hoc addition to remove asap                
            tra_loader = torch.utils.data.DataLoader(create_dataset_MLP_from_graphs(tra_dataset), 
                                                     batch_size=batch_size, shuffle=True, pin_memory=True)
            val_loader = torch.utils.data.DataLoader(create_dataset_MLP_from_graphs(val_dataset), 
                                                     batch_size=batch_size, shuffle=False, pin_memory=True)
            tst_loader = torch.utils.data.DataLoader(create_dataset_MLP_from_graphs(tst_dataset), 
                                                     batch_size=batch_size, shuffle=False, pin_memory=True)

        else:
            # it's a GNN
            tra_loader = torch_geometric.loader.DataLoader(tra_dataset, batch_size=batch_size, 
                                                           shuffle=True, pin_memory=True)
            val_loader = torch_geometric.loader.DataLoader(val_dataset, batch_size=batch_size, 
                                                           shuffle=False, pin_memory=True)                
            tst_loader = torch_geometric.loader.DataLoader(tst_dataset, batch_size=batch_size, 
                                                           shuffle=False, pin_memory=True)

        
        # create results dataframe
        results_df = pd.DataFrame(list(all_combinations))
        results_df = pd.concat([results_df,
                                pd.DataFrame(index=np.arange(len(all_combinations)), 
                                          columns=list(res_columns))],axis=1)        
        
        for i, combination in enumerate(all_combinations):
            if i<18:
                print(f'i={i}; skipping')
                continue
            # TO DO: This hardcoded if/else below ain't good --> let's change it
            if algorithm == 'MLP':
                for temp in tra_loader:
                    break
                combination['num_inputs']  = temp[0].shape[1]  # all this ad-hoc stuff must be removed
                combination['num_outputs'] = temp[1].shape[1]  # (e.g., these lines are needed to instantiate MLP properly)            
            else:
                combination['edge_features']=edge_size # all this ad-hoc stuff must be removed
                combination['node_features']=node_size # (e.g., these two lines are needed to instantiate GNN properly)
            
            print(f'{algorithm}: training combination {i+1} of {len(all_combinations)}\t',end='\r',)    

            # model creation
            model = getattr(sys.modules[__name__], algorithm)(**combination).to(device)                        
            total_parameters = sum(p.numel() for p in model.parameters())
            
            # model optimizer
            optimizer = optim.Adam(params=model.parameters(), **cfg['adamParams'])            

            # training
            model, tra_losses, val_losses, elapsed_time = training(model, optimizer, tra_loader, val_loader,
                                                                   n_epochs=num_epochs, patience=10, report_freq=0, 
                                                                   alpha=alpha, lr_rate=5, lr_epoch=50, 
                                                                   normalization=None)
            # store training history and model
            pd.DataFrame(data = np.array([tra_losses, val_losses]).T).to_csv(
                f'{results_folder}/{wdn}/{algorithm}/hist/{i}.csv')
            torch.save(model, f'{results_folder}/{wdn}/{algorithm}/models/{i}.csv')
                         
            # compute and store predictions, compute r2 scores        
            losses = {}
            r2_scores = {}            
            for split, loader in zip(['training','validation','testing'],[tra_loader,val_loader,tst_loader]):
                losses[split], pred, real = testing(model, loader)
                r2_scores[split] = r2_score(real, pred)
                if i == 0:
                    pd.DataFrame(data=real.reshape(-1,n_nodes)).to_csv(
                        f'{results_folder}/{wdn}/{algorithm}/pred/{split}/real.csv') # save real obs
                pd.DataFrame(data=pred.reshape(-1,n_nodes)).to_csv(
                    f'{results_folder}/{wdn}/{algorithm}/pred/{split}/{i}.csv')
                
            # store results
            results_df.loc[i,res_columns] = (losses['training'], losses['validation'], losses['testing'], 
                                             r2_scores['training'], r2_scores['validation'], r2_scores['testing'], 
                                             total_parameters, elapsed_time, num_epochs)
        # save graphnormalizer
        with open(f'{results_folder}/{wdn}/{algorithm}/gn.pickle', 'wb') as handle:
            pickle.dump(gn, handle, protocol=pickle.HIGHEST_PROTOCOL)
        results_df.to_csv(f'{results_folder}/{wdn}/{algorithm}/results.csv') 


Working with FOS, network 1 of 6
i=0; skipping
i=1; skipping
i=2; skipping
i=3; skipping
i=4; skipping
i=5; skipping
i=6; skipping
i=7; skipping
i=8; skipping
i=9; skipping
i=10; skipping
i=11; skipping
i=12; skipping
i=13; skipping
i=14; skipping
i=15; skipping
i=16; skipping
i=17; skipping


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/bulat/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_13655/2832782963.py", line 85, in <module>
    model, tra_losses, val_losses, elapsed_time = training(model, optimizer, tra_loader, val_loader,
  File "/home/bulat/PycharmProjects/gnn_wds_deploy/training/train.py", line 117, in training
    train_loss = train_epoch(model, train_loader, optimizer, alpha=alpha, normalization=normalization, device=device)
  File "/home/bulat/PycharmProjects/gnn_wds_deploy/training/train.py", line 47, in train_epoch
    preds = model(batch)
  File "/home/bulat/miniconda3/envs/graphenv38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_13655/2584101711.py", line 80, in forward
    x = self.convs[i](x=x, edge_index=edge_index)
  File "/home/

TypeError: object of type 'NoneType' has no len()